## Stream attributes using Signals

This notebook creates a new view using the SDK that will be computed using stream processing.

### Flow of data

```mermaid
flowchart LR
    sp(Snowplow Pipeline)
    stream[/Stream processing/]
    signals(Signals)

    sp --> stream
    stream --> signals
```

---

# Installation and setup

In [ ]:
%pip install snowplow-signals

In [ ]:
from snowplow_signals import Signals
from dotenv import load_dotenv
import os

load_dotenv()

sp_signals = Signals(
    api_url=os.environ["SNOWPLOW_API_URL"],
    api_key=os.environ["SNOWPLOW_API_KEY"],
    api_key_id=os.environ["SNOWPLOW_API_KEY_ID"],
    org_id=os.environ["SNOWPLOW_ORG_ID"],
)

### Define a new attribute

This block creates a single attribute definition including the logic how it should be calculated (it's filters and aggregation).

The attribute calculates the number of add to cart ecommerce events.

In [ ]:
from snowplow_signals import Attribute, Criteria, Criterion, Event

products_added_to_cart = Attribute(
    name="products_added_to_cart",
    type="string_list",
    events=[
        Event(
            vendor="com.snowplowanalytics.snowplow.ecommerce",
            name="snowplow_ecommerce_action",
            version="1-0-2",
        )
    ],
    aggregation="unique_list",
    property="contexts_com_snowplowanalytics_snowplow_ecommerce_product_1[0].name",
    criteria=Criteria(
        all=[
            Criterion(
                property="unstruct_event_com_snowplowanalytics_snowplow_ecommerce_snowplow_ecommerce_action_1:type",
                operator="=",
                value="add_to_cart",
            ),
        ],
    ),
)

### Wrapping the attribute in a view

All attributes need to be included in views that can be considered as "tables" of attributes.

Views are immutable and versioned.

In [10]:
from snowplow_signals import View, session_entity

view = View(
    name="my_ecommerce_attributes",
    version=1,
    entity=session_entity,
    attributes=[
        products_added_to_cart,
    ],
)

### Applying the view to Signals

The following block pushes the view definition to the Signals API and makes it available for processing.

In [ ]:
from snowplow_signals import Signals

applied = sp_signals.apply([view])
print(f"{len(applied)} objects applied")

### Retrieving data

One can fetch the latest attribute values for a particular session from the view as follow.

In [ ]:
response = sp_signals.get_online_attributes(
    view,
    "c5b91072-06b3-4aa7-a558-8e11dcc09993",
)

response.to_dataframe()